In [ ]:
import tensorflow as tf

In [ ]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train, X_test = X_train/255.0, X_test/255.0

In [ ]:
mnistnn = tf.keras.models.Sequential([
               tf.keras.layers.Flatten(input_shape=(28,28,1)),                       
               tf.keras.layers.Dense(units=128, activation='relu'),    
               tf.keras.layers.Dense(units=10)                   
])

In [ ]:
mnistnn.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='sgd', metrics=['accuracy'])

In [ ]:
history = mnistnn.fit(x=X_train, y=y_train,
                      epochs=10, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Validation accuracy')
plt.legend(['Train','Validation'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Validation loss')
plt.legend(['Train','Validation'])
plt.show()

In [ ]:
mnistnn.evaluate(X_test, y_test)

In [ ]:
mnistnn.save('mnistnn.h5', save_format='h5')

In [ ]:
from keras.models import load_model

newmodel = load_model('mnistnn.h5')

In [ ]:
newmodel.summary()

In [ ]:
newmodel.evaluate(X_test, y_test)

In [ ]:
ypred = mnistnn.predict_classes(X_test)

#Deployment

In [ ]:
!pip install flask gevent requests pillow

Creat a file ProcFile

In [ ]:
procfile = 'web: gunicorn app:app'
procfiles= open("/content/Procfile","w")
procfiles.write(procfile)
procfiles.close()

#Install flask and ngrok

In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask

#Webpage

In [ ]:
a = '''
<!doctype html>
<html lang="en">
<head>
</head> 
  <body>
 
          <h3>Image Recognition Server</h3>
       
      <form action="" method=post enctype=multipart/form-data>
        <input type=file name=file>
        <input type=submit value=Upload>
    </form>
    {{label}}  
     
  </body>
  </html>
  '''

In [ ]:
!mkdir '/content/templates'
!mkdir '/content/uploads'

Html_file = open("/content/templates/index.html", "w")
Html_file.write(a)
Html_file.close()

#Deploy model

In [ ]:
import os
from flask import Flask, render_template, request
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array

app = Flask(__name__)
run_with_ngrok(app)
app.config['UPLOADS'] = 'uploads'


def load_mymodel():
    global mymodel
    mymodel = load_model('mnistnn.h5')

@app.route('/')
def home():
    return render_template('index.html')

def predictions(file):
    img = image.load_img(file, target_size=(28,28), color_mode="grayscale")
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    ans = mymodel.predict_classes(img)
    return ans


@app.route('/', methods=['POST'])
def upload_files():
    file = request.files['file']
    filepath = os.path.join(app.config['UPLOADS'], file.filename)
    file.save(filepath)
    output = predictions(filepath)
    return render_template('index.html', label=output)



if __name__=='__main__':
    load_mymodel()
    app.run()